In [1]:
import random
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.metrics import mean_absolute_error
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger # Callback
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger # Callback
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/dhamvi01/Univariate-Time-Series-using-LSTM/master/airline-passengers.csv")
data

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [3]:
passengers=data['Passengers'].values
passengers

array([112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118, 115,
       126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140, 145, 150,
       178, 163, 172, 178, 199, 199, 184, 162, 146, 166, 171, 180, 193,
       181, 183, 218, 230, 242, 209, 191, 172, 194, 196, 196, 236, 235,
       229, 243, 264, 272, 237, 211, 180, 201, 204, 188, 235, 227, 234,
       264, 302, 293, 259, 229, 203, 229, 242, 233, 267, 269, 270, 315,
       364, 347, 312, 274, 237, 278, 284, 277, 317, 313, 318, 374, 413,
       405, 355, 306, 271, 306, 315, 301, 356, 348, 355, 422, 465, 467,
       404, 347, 305, 336, 340, 318, 362, 348, 363, 435, 491, 505, 404,
       359, 310, 337, 360, 342, 406, 396, 420, 472, 548, 559, 463, 407,
       362, 405, 417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390,
       432])

In [4]:
def split_sequence(sequence, n_steps_in):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in

		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
# define univariate time series
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(series.shape)
# transform to a supervised learning problem
X, y = split_sequence(passengers, 2)
print(X.shape, y.shape)

dfX = pd.DataFrame(X, columns=["Xt-2", "Xt-1"])
dfy = pd.DataFrame(y, columns=["Xt"])

df = pd.concat((dfX, dfy), axis = 1)
df

(10,)
(142, 2) (142,)


,Xt-2,Xt-1,Xt
0,112,118,132
1,118,132,129
2,132,129,121
3,129,121,135
4,121,135,148
...,...,...,...
137,535,622,606
138,622,606,508
139,606,508,461
140,508,461,390


data aktual/prediksi

In [5]:
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [6]:
t =  data.iloc[: , [ 1]] 
#define the input sequence
raw_seq = t.values
raw_seq_train = raw_seq[:428]

# choose a number of time steps
n_steps_in, n_steps_out = 3,3
# split into samples
X, y = split_sequence(raw_seq_train, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))


In [7]:
X
# 5,6,3,1,8,2,4,7
#17,9,2,18,5,13,17

array([[[112],
        [118],
        [132]],

       [[118],
        [132],
        [129]],

       [[132],
        [129],
        [121]],

       [[129],
        [121],
        [135]],

       [[121],
        [135],
        [148]],

       [[135],
        [148],
        [148]],

       [[148],
        [148],
        [136]],

       [[148],
        [136],
        [119]],

       [[136],
        [119],
        [104]],

       [[119],
        [104],
        [118]],

       [[104],
        [118],
        [115]],

       [[118],
        [115],
        [126]],

       [[115],
        [126],
        [141]],

       [[126],
        [141],
        [135]],

       [[141],
        [135],
        [125]],

       [[135],
        [125],
        [149]],

       [[125],
        [149],
        [170]],

       [[149],
        [170],
        [170]],

       [[170],
        [170],
        [158]],

       [[170],
        [158],
        [133]],

       [[158],
        [133],
        [114]],

       [[133]